In [1]:
import blocksci
import matplotlib.pyplot as plt
import matplotlib.ticker
import collections
import pandas as pd
import numpy as np
import csv
import datetime
%matplotlib notebook

In [2]:
# important functions for the rates
folder_name = "bitcoin_fiat_analysis/more_rates/"
# exchange_files = ["btc-aud-max.csv","btc-brl-max.csv","btc-cad-max.csv","btc-chf-max.csv","btc-cny-max.csv",
#                  "btc-dkk-max.csv","btc-eur-max.csv","btc-gbp-max.csv","btc-hkd-max.csv","btc-idr-max.csv",
#                  "btc-inr-max.csv","btc-jpy-max.csv","btc-krw-max.csv","btc-mxn-max.csv","btc-myr-max.csv",
#                  "btc-nzd-max.csv","btc-php-max.csv","btc-pln-max.csv","btc-rub-max.csv","btc-sek-max.csv",
#                  "btc-sgd-max.csv","btc-twd-max.csv","btc-usd-max.csv","btc-zar-max.csv"]
exchange_files = ["btc-usd-max.csv"]

def get_rates_data():
    rates = {}
    for rate in exchange_files:
        this_rate = pd.read_csv(folder_name+rate)
        rates[rate[:7]] = this_rate
    return rates

# rates variable is defined from get_rates_data()
dollar_df = pd.read_csv(folder_name+exchange_files[0])
# def get_price(date, currency_df):
def get_price(date):
#     price = currency_df.loc[currency_df['snapped_at'] == (date + " 00:00:00 UTC")]['price']
    price = dollar_df.loc[dollar_df['snapped_at'] == (date + " 00:00:00 UTC")]['price']
    if len(price)>0:
        return float(price)
    else:
        return "no info for date"

In [3]:
# parser_data_directory should be set to the data-directory which the blocksci_parser output
chain = blocksci.Blockchain("/home/avery/Developer/blocksci-data")

In [4]:
type(chain)

blocksci.blocksci_interface.Blockchain

In [5]:
first_block = chain[0]

In [6]:
first_block.timestamp

1231006505

In [7]:
first_block.outputs.value

[5000000000]

In [7]:
fees = [sum(block.txes.fee) / len(block) for block in chain.range('2017')]

In [8]:
len(fees)

55931

In [4]:
blocks_1418 = [block for block in chain.range('12-04-2017','12-09-2017')]

In [5]:
# blocks_2017 = [block for block in chain.range('2017')]
len(blocks_1418)

850

In [7]:
total_txns = 0
for block in blocks_1418:
    total_txns += len(block.txes)
total_txns

2023732

In [8]:
# initialize all vars for the analysis
# this array will hold all of the processed transactions
inferred_transactions = np.array([])
rates = get_rates_data();

In [7]:
blocks_1418[0].txes.all[0].block_time # already a datetime
blocks_1418[0].txes.all[0].outputs[0]

TxOut(spending_tx_index=278113155, address=PubkeyHashAddress(1CK6KHY6MHgYvmRQ4PAafKYDrg1ejbH1cE), value=1297127127)

In [13]:
inferred_transactions = []

In [6]:
satoshi_to_btc = 0.00000001 # one one-hundredth million of a bitcoin is a Satoshi
def infer_currency(txn):
    best_error = 10**9
    inferred_currency = 'none'
    for output in txn.outputs:
        for rate in rates.keys():
            date = txn.block_time.strftime('%Y-%m-%d')
            exchange_rate = get_price(date, rates[rate])
#             print(exchange_rate, type(exchange_rate), 'exchange_rate')
            txn_amount = exchange_rate*satoshi_to_btc*output.value
            diff = abs(txn_amount - int(round(txn_amount)))
            if diff < best_error:
                best_error = diff
                inferred_currency = rate
    return inferred_currency

In [29]:
satoshi_to_btc = 0.00000001 # one one-hundredth million of a bitcoin is a Satoshi
def infer_dollar(txn):
    dollar = False
    date = txn.block_time.strftime('%Y-%m-%d')
    exchange_rate = get_price(date)
    for output in txn.outputs:
        txn_amount = int(exchange_rate*satoshi_to_btc*output.value) # rounding to int so as to get rid of cents
        if (txn_amount % 100 == 0) or (txn_amount % 10 == 0) or (txn_amount % 5 == 0):
            us_lower = 4
            us_higher = 16
            if txn.block_time.hour > us_lower and txn.block_time.hour < us_higher: 
                return True
            # if any of the outputs is a whole dollar value
    return False
#         diff = abs(txn_amount - int(round(txn_amount)))
#     if diff < 0.5:
#         return True
#     return False

In [ ]:
import time
start = time.time()
inferred_transactions = []
block_count = 0 # out of 55931 or 850 (1418)
for block in blocks_1418:
    block_count += 1
    if block_count % 50 == 0:
        print(block_count)
    for txn in block.txes.all:
        inferred_transactions.append(infer_dollar(txn))
#         np.append(inferred_transactions,infer_currency(txn))

end = time.time()
print(end - start)

50
100
150
200
250
300
350
400
450
500
550
600
650
700


In [20]:
chinese_inferred = []
chinese_df = pd.read_csv(folder_name+"btc-cny-max.csv")
# def get_price(date, currency_df):
def get_rmb(date):
#     price = currency_df.loc[currency_df['snapped_at'] == (date + " 00:00:00 UTC")]['price']
    price = chinese_df.loc[chinese_df['snapped_at'] == (date + " 00:00:00 UTC")]['price']
    if len(price)>0:
        return float(price)
    else:
        return "no info for date"
satoshi_to_btc = 0.00000001 # one one-hundredth million of a bitcoin is a Satoshi
def infer_rmb(txn):
    date = txn.block_time.strftime('%Y-%m-%d')
    exchange_rate = get_rmb(date)
    for output in txn.outputs:
        txn_amount = int(exchange_rate*satoshi_to_btc*output.value) # rounding to int so as to get rid of cents
        if (txn_amount % 100 == 0) or (txn_amount % 10 == 0) or (txn_amount % 5 == 0):
            # if any of the outputs is a whole dollar value
            china_lower = 1
            china_higher = 15
            if txn.block_time.hour > china_lower and txn.block_time.hour < china_higher: 
                return True
    return False

In [ ]:
import time
start = time.time()
chinese_inferred = []
block_count = 0 # out of 55931 or 850 (1418)
for block in blocks_1418:
    block_count += 1
    if block_count % 50 == 0:
        print(block_count)
    for txn in block.txes.all:
        chinese_inferred.append(infer_rmb(txn))
#         np.append(inferred_transactions,infer_currency(txn))

end = time.time()
print(end - start)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800


In [23]:
yen_df = pd.read_csv(folder_name+"btc-jpy-max.csv")
# def get_price(date, currency_df):
def get_yen(date):
#     price = currency_df.loc[currency_df['snapped_at'] == (date + " 00:00:00 UTC")]['price']
    price = yen_df.loc[yen_df['snapped_at'] == (date + " 00:00:00 UTC")]['price']
    if len(price)>0:
        return float(price)
    else:
        return "no info for date"
satoshi_to_btc = 0.00000001 # one one-hundredth million of a bitcoin is a Satoshi
def infer_yen(txn):
    date = txn.block_time.strftime('%Y-%m-%d')
    exchange_rate = get_yen(date)
    for output in txn.outputs:
        txn_amount = int(exchange_rate*satoshi_to_btc*output.value) # rounding to int so as to get rid of cents
        if (txn_amount % 100 == 0) or (txn_amount % 10 == 0) or (txn_amount % 5 == 0):
            yen_lower = 0
            yen_higher = 14
            if txn.block_time.hour > yen_lower and txn.block_time.hour < yen_higher: 
                return True
    return False


start = time.time()
japanese_inferred = []
block_count = 0
for block in blocks_1418:
    block_count += 1
    if block_count %50 == 0:
        print(block_count)
    for txn in block.txes.all:
        japanese_inferred.append(infer_yen(txn))

end = time.time()
print(end - start)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
758.638085603714


In [31]:
won_df = pd.read_csv(folder_name+"btc-krw-max.csv")
# def get_price(date, currency_df):
def get_won(date):
#     price = currency_df.loc[currency_df['snapped_at'] == (date + " 00:00:00 UTC")]['price']
    price = won_df.loc[won_df['snapped_at'] == (date + " 00:00:00 UTC")]['price']
    if len(price)>0:
        return float(price)
    else:
        return "no info for date"
satoshi_to_btc = 0.00000001 # one one-hundredth million of a bitcoin is a Satoshi
def infer_won(txn):
    date = txn.block_time.strftime('%Y-%m-%d')
    exchange_rate = get_won(date)
    for output in txn.outputs:
        txn_amount = int(exchange_rate*satoshi_to_btc*output.value) # rounding to int so as to get rid of cents
        if (txn_amount % 100 == 0) or (txn_amount % 10 == 0):
            won_lower = 0
            won_higher = 14
            if txn.block_time.hour > won_lower and txn.block_time.hour < won_higher: 
                return True
    return False


start = time.time()
won_inferred = []
block_count = 0
for block in blocks_1418:
    block_count += 1
    if block_count %50 == 0:
        print(block_count)
    for txn in block.txes.all:
        won_inferred.append(infer_won(txn))

end = time.time()
print(end - start)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
776.7927088737488


In [32]:
print('chinese %' + str(sum(chinese_inferred)/len(chinese_inferred)))
print('japanese %' + str(sum(japanese_inferred)/len(japanese_inferred)))
print('dollar %' + str(sum(inferred_transactions)/len(inferred_transactions)))
print('won %' + str(sum(won_inferred)/len(won_inferred)))

chinese %0.20550398965870975
japanese %0.19906884903732314
dollar %0.18305190608242594
won %0.1214572878226959


In [19]:
type(blocks_1418[0].txes.all[0].block_time.hour)

int

In [ ]:
for block in blocks_1418:
    block_count += 1
    if block_count %50 == 0:
        print(block_count)
    for txn in block.txes.all:
#         japanese_inferred.append(infer_yen(txn))

In [48]:
# started at 4:49am.
blocks_2017[0].coinbase_tx.output_value

1260658449

In [63]:
# script to see how much money miners made in 2017
miners_total = 0
for block in blocks_2017:
    miners_total += block.fee + block.coinbase_tx.output_value
miners_total*satoshi_to_btc

In [6]:
count = 0 
for block in blocks_2017:
    for txn in block.txes.all:
        if blocksci.blocksci_interface.heuristics.is_definite_coinjoin(txn):
            count += 1
count

TypeError: is_definite_coinjoin(): incompatible function arguments. The following argument types are supported:
    1. (arg0: blocksci.blocksci_interface.Tx, arg1: int, arg2: float, arg3: int) -> blocksci::heuristics::CoinJoinResult
    2. (arg0: blocksci.blocksci_interface.Tx, arg1: int, arg2: float) -> blocksci::heuristics::CoinJoinResult

Invoked with: Tx(len(txins)=0, len(txouts)=1, size_bytes=128, block_height=446064, tx_index=183813678)

In [10]:
hi = blocksci.blocksci_interface.heuristics.coinjoin_txes(chain,0,1000)